In [1]:
import pandas as pd
import subprocess

In [2]:
def generate_workspace():
    subprocess.run(["mkdir", "-p", "tmp_dir"])

In [3]:
import os


def cut_video(data:pd.Series, tmp_dir:str):
    """
    A function that cuts video according to pd.Series values.
    This function will execute following command using subprocess.
    ffmpeg -ss 00:01:00 -to 00:02:00  -i input.mp4 -c copy output.mp4
    
    :param data: a pandas.Series that represents the data of current video clip
    :param tmp_dir: a string that represents tmp_dir
    """
    start_str = data['start']
    end_str = data['end']
    country_name = data['name']
    save_name = data['save_name']
    out_file_name = os.path.join(tmp_dir, save_name)
    subprocess.check_output([
        "ffmpeg", 
        "-ss", 
        start_str, 
        "-to",
        end_str, 
        "-i", 
        "../OriginalVideo.mp4",
        "-c", 
        "copy", 
        out_file_name, "-y"], 
        stderr=subprocess.STDOUT)

In [101]:
def change_video_speed(video_name:str, speed:float):
    """
A function that changes video speed into designated speed.
    This will generate a video with %03ds.mp4.
    Example, if 000.mp4 was processed, it will be saved as 000s.mp4.
    
    The problem with ffmpeg is that ffmpeg can only go as low as 0.5.
    So in order for us to achieve lower speed than 0.5, it means we have to process it multiple times.
    For example, we have 0.123 as speed.
    
    This means the following: 0.123 = 0.5 * 0.5 * 0.5 * 0.984.
    So, we need to make the video clip go 0.5, then that video clip goes 0.5, and ...
    
    This function will execute following command using subprocess.
    ffmpeg -i Break0.mp4 -vf "setpts=(PTS-STARTPTS)/2.5" -af atempo=2.5 a.mp4

    :param video_name: a string that represents video's name to process.
    :param speed: a float value that represents how fast the video should go.
    """
    if speed < 0.5:  # if speed was 0.5, we need to slow it multiple times
        cur_speed = speed
        multiplier_list = list()  # for storing all elements for making the wanted speed
        while (cur_speed < 0.5):
            cur_speed = cur_speed / 0.5
            multiplier_list.append(0.5)
        multiplier_list.append(cur_speed)         
        # The formula will be like this
        # 1. Make 0.5 times slower and name it s_0.mp4
        # 2. s_i would be s_i-1 slowed in the speed_i
        # 3. Iterate it over and over
        
        operation_count = 0  # count operations
        # Make s_0.mp4 from original video
        subprocess.check_output([
                "ffmpeg",
                "-i",
                video_name,
                "-vf",
                "setpts=(PTS-STARTPTS)/%f" % 0.5,
                "-af",
                "atempo=%f" % 0.5,
                video_name.replace(".mp4", "s_0.mp4"),
                "-y"],
                stderr=subprocess.STDOUT)
        operation_count += 1
        
        for speed in multiplier_list[1:]:  # iterate and slow s_i-1 
            subprocess.check_output([
            "ffmpeg",
            "-i",
            video_name.replace(".mp4", "s_%d.mp4" % (operation_count - 1)),
            "-vf",
            "setpts=(PTS-STARTPTS)/%f" % speed,
            "-af",
            "atempo=%f" % speed,
            video_name.replace(".mp4", "s_%d.mp4" % operation_count),
            "-y"],
            stderr=subprocess.STDOUT)
            operation_count += 1
            # When done slowing, delete the old version s_i-1
            #os.remove(video_name.replace(".mp4", "s_%d.mp4" % (operation_count - 1)))
        # Save the final video as s.mp4
        os.rename(video_name.replace(".mp4", "s_%d.mp4" % (operation_count - 1)), video_name.replace(".mp4", "s.mp4"))
    else:  # if speed was faster than 0.5, just apply the speed
        subprocess.check_output([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "setpts=(PTS-STARTPTS)/%f" % speed,
        "-af",
        "atempo=%f" %speed,
        video_name.replace(".mp4", "s.mp4"),
        "-y"],
        stderr=subprocess.STDOUT)             

In [103]:
def add_caption(video_name:str, country_name:str, value:str, 
                font_dir:str, font_color:str, font_size:int,
                box_enabled:bool, box_opacity:float, box_color:str, box_padding:int,
                text_position:tuple):
    """
    A function that adds caption to the video.
    This will generate a video with %03df.mp4.
    Example, if 000s.mp4 was processed, it will be saved as 000f.mp4.
    
    This function will execute following command using subprocess.
    ffmpeg -i 000s.mp4 -vf "drawtext=fontfile=/path/to/font.ttf:text='Stack Overflow':fontcolor=white:fontsize=24:box=1:boxcolor=black@0.5:boxborderw=5:x=(w-text_w)/2:y=(h-text_h)/2" -codec:a copy 000f.mp4

    :param video_name: a string that represents video's name to process.
    :param country_name: a string that represents country's name.
    :param value: a string that represents value of that country.
    :param font_dir: a string that represents directory that leads to the font that current caption will be using.
    :param font_color: a string that represents color Ex) white.
    :param font_size: a int value that represents font size.
    :param box_enabled: a boolean that represents whether or not to use text boxes.
    :param box_opacity: a float that sets box's opacity: 0 means it is it is not transparent at all.
    :param box_color: a string that represents box's color.
    :param box_padding: a string that adds padding to the text box.
    :param text_position: a tuple of text that represents where to locate texts.
    """
    text = country_name + "  " + value
    text_position_x = ""
    text_position_y = ""
    
    # set text_position_x using three options
    if text_position[0] == "center":  # when x position was center
        text_position_x = "(w-text_w)/2"  
    elif text_position[0] == "left":  # when x position was left
        text_position_x = "(%d)" % box_padding      
    elif text_position[0] == "right":  # when x position was right
        text_position_x = "(w-text_w-%d)" % box_padding
    else:  # if value was other than this, raise ValueError
        raise ValueError
    
    # set text_position_y using three options
    if text_position[1] == "center":  # when y position was center
        text_position_y = "(h-text_h)/2"
    elif text_position[1] == "up":  # when y position was up
        text_position_y = "(%d)" % box_padding
    elif text_position[1] == "down":  # when y position was down
        text_position_y = "(h-text_h-%d)" % box_padding
    else:  # if value was other than this, raise ValueError
        raise ValueError
    
    subprocess.check_output([
        "ffmpeg",
        "-i",
        video_name,
        "-vf",
        "drawtext=fontfile=/%s:text='%s':fontcolor=%s:fontsize=%d:box=%d:boxcolor=%s@%f:boxborderw=5:x=%s:y=%s" 
        % (font_dir, text, font_color, font_size, box_enabled, box_color, box_opacity, text_position_x, text_position_y),
        "-codec:a",
        "copy",
        video_name.replace("s.mp4", "f.mp4"),
        "-y"],
        stderr=subprocess.STDOUT)

In [104]:
def calculate_play_speed(cur_value, avg, bigger_the_faster:bool):
    """
    A simple function that calculates playspeed of current segment of the video.
    This will range speed from 0.xx to 2.0
    :param cur_value: the current value of this segment
    :param avg: the average value of total segments
    :param bigger_the_faster: if true it will be bigger the faster
    """
    return  (1 + (-1) ** (not bigger_the_faster) * ((cur_value - avg) / avg))

In [105]:
def process_country(**kwargs):
    """
    A function that process a single country.
    This function does following things
    1. Cut video clip according to the timings.
    2. Calculate playspeed.
    3. Set playspeed into the video.
    4. Add caption.
    :param data: a pandas.Series that represents the data of current video clip
    :param cur_value: value that representes current country's value
    :param playspeed: value that represents manual playspeed
    :param tmp_dir: a string that represents tmp_dir
    :param auto_calculate_speed: a boolean that decides whether or not to calculate speed.
    :param bigger_the_faster: a boolean that decides bigger the faster for playspeed calculation
    :param avg: a value that is for average of all countries.
    :param font_dir: a string that represents directory that leads to the font that current caption will be using.
    :param font_color: a string that represents color Ex) white.
    :param font_size: a int value that represents font's size.
    :param box_enabled: a boolean that represents whether or not to use text boxes.
    :param box_opacity: a float that sets box's opacity: 0 means it is it is not transparent at all.
    :param box_color: a string that represents box's color.
    :param box_padding: a string that adds padding to the text box.
    :param text_position: a tuple of text that represents where to locate texts.
    """
    data = kwargs['data']
    cur_value = kwargs['cur_value']
    playspeed = kwargs['playspeed']
    
    tmp_dir = kwargs['tmp_dir']
    auto_calculate_speed = kwargs['auto_calculate_speed']
    bigger_the_faster = kwargs['bigger_the_faster']
    avg = kwargs['avg']
    font_dir = kwargs['font_dir']
    font_color = kwargs['font_color']
    font_size = kwargs['font_size']
    box_enabled = kwargs['box_enabled']
    box_opacity = kwargs['box_opacity']
    box_color = kwargs['box_color']
    box_padding = kwargs['box_padding']
    text_position = kwargs['text_position']
    
    out_file_name = os.path.join(tmp_dir, data['save_name'])
    
    print("[+] Processing " + data['name'])
    
    if (playspeed == -1):  # if playspeed was -1, this will skip current country
        print("[+] Skipping " + data['name'])
        return
    else:
        cut_video(data, tmp_dir)
        if auto_calculate_speed:
            playspeed = calculate_play_speed(cur_value, avg, bigger_the_faster)

        change_video_speed(out_file_name, playspeed)
        add_caption(video_name=out_file_name.replace(".mp4", "s.mp4"), 
                    country_name=data['name'], value=str(cur_value), 
                    font_dir=font_dir, font_color=font_color, font_size=font_size, 
                    box_enabled=box_enabled, box_opacity=box_opacity, box_color=box_color, box_padding=box_padding, 
                    text_position=text_position)

In [106]:
def process_all(**kwargs):
    """
    :param data: a pandas.Series that represents the data of current video clip
    :param cur_value: value that representes current country's value
    :param tmp_dir: a string that represents tmp_dir
    :param bigger_the_faster: a boolean that decides bigger the faster for playspeed calculation
    :param avg: a value that is for average of all countries.
    :param font_dir: a string that represents directory that leads to the font that current caption will be using.
    :param font_color: a string that represents color Ex) white.
    :param font_size: a int value that represents font's size.
    :param box_enabled: a boolean that represents whether or not to use text boxes.
    :param box_opacity: a float that sets box's opacity: 0 means it is it is not transparent at all.
    :param box_color: a string that represents box's color.
    :param box_padding: a string that adds padding to the text box.
    :param text_position: a tuple of text that represents where to locate texts.
    """

In [107]:
if __name__ == "__main__":
    generate_workspace()
    tmp_dir = os.path.join(os.getcwd(), "tmp_dir")
    audio_dir = os.path.join(tmp_dir, "audio")
    data = pd.read_csv("../time_data.csv")
    process_country(data=data.loc[1], cur_value=2, playspeed=0.123, tmp_dir=tmp_dir, auto_calculate_speed=False, bigger_the_faster=True, avg=1, font_dir="../fonts/comic.ttf",
                    font_color="white", font_size=40, box_enabled=True, box_opacity=0.5, box_color="black", box_padding=10, text_position=('left', 'down'))

[+] Processing United States
[0.5, 0.5, 0.5, 0.984]


In [108]:
from IPython.display import Video

Video("./tmp_dir/001f.mp4")

In [9]:
Video("./tmp_dir/000f.mp4")